In [1]:
import numpy as np
import pandas as pd 

In [2]:
# Load datasets
zomato = pd.read_csv("zomato.csv", encoding="latin-1")
cities = pd.read_csv("cities.csv")

In [3]:
cities.head()

,Rank,City,Population(2001),State or union territory
0,1,Mumbai,"1,19,78,450",Maharashtra
1,2,Delhi,"98,79,172",Delhi
2,3,Bangalore,"43,01,326",Karnataka
3,4,Hyderabad,"36,37,483",Telangana
4,5,Ahmedabad,"35,20,085",Gujarat


In [4]:
cities = cities.rename(columns={
    "City": "city",
    "Population(2001)": "population",
    "State or union territory": "state"
})


In [5]:
cities["population"] = pd.to_numeric(
    cities["population"].astype(str).str.replace(",", "", regex=False),
    errors='coerce'
)


In [6]:
cities.sample(3)

,Rank,city,population,state
45,46,Kota[6],694316.0,Rajasthan
267,268,Machilipatnam,179353.0,Andhra Pradesh
73,74,Cuttack,534654.0,Odisha


In [7]:
import re

city_mapping = {
    "bengaluru": "bangalore",
    "bangalore urban": "bangalore",
    "vasco da gama": "vasco",
    "new delhi": "delhi",
    "delhi ncr": "delhi",
    "bengaluru": "bangalore",
    "bangalore urban": "bangalore",
    "gurugram": "gurgaon",
    "gurgaon": "gurgaon",
    "navi mumbai": "mumbai",
    "mumbai suburban": "mumbai"
}

def clean_city_name(name):
    if pd.isna(name):
        return None

    name = name.lower().strip()

    # remove footnotes like [35]
    name = re.sub(r"\[.*?\]", "", name)

    # remove special characters
    name = re.sub(r"[^a-z\s]", "", name)

    # normalize spaces
    name = re.sub(r"\s+", " ", name).strip()

    # map to standard name
    return city_mapping.get(name, name)

cities["city_clean"] = cities["city"].apply(clean_city_name)

In [8]:
cities.sample(5)

,Rank,city,population,state,city_clean
18,19,Patna,1366444.0,Bihar,patna
16,17,Visakhapatnam[4],1345938.0,Andhra Pradesh,visakhapatnam
271,272,Udupi,NaN,Karnataka,udupi
82,83,Nanded,430733.0,Maharashtra,nanded
268,269,Shimla,142555.0,Himachal Pradesh,shimla


In [9]:
def city_tier(pop):
    if pd.isna(pop):
        return "Unknown"
    elif pop >= 1_000_000:
        return "Tier 1"
    elif pop >= 500_000:
        return "Tier 2"
    else:
        return "Tier 3"

cities["city_tier"] = cities["population"].apply(city_tier)

In [10]:
cities.tail()

,Rank,city,population,state,city_clean,city_tier
317,318,Tezpur,83028.0,Assam,tezpur,Tier 3
318,319,Jehanabad[35],81503.0,Bihar,jehanabad,Tier 3
319,320,Aurangabad[36],79393.0,Bihar,aurangabad,Tier 3
320,321,Gangtok,29354.0,Sikkim,gangtok,Tier 3
321,322,Vasco Da Gama,NaN,Goa,vasco,Unknown


In [11]:
cities["state_clean"] = (
    cities["state"].str.lower().str.strip()
)

In [12]:
cities.tail()

,Rank,city,population,state,city_clean,city_tier,state_clean
317,318,Tezpur,83028.0,Assam,tezpur,Tier 3,assam
318,319,Jehanabad[35],81503.0,Bihar,jehanabad,Tier 3,bihar
319,320,Aurangabad[36],79393.0,Bihar,aurangabad,Tier 3,bihar
320,321,Gangtok,29354.0,Sikkim,gangtok,Tier 3,sikkim
321,322,Vasco Da Gama,NaN,Goa,vasco,Unknown,goa


In [13]:
cities["city_clean"].value_counts()

city_clean
mumbai            2
aurangabad        2
delhi             2
kolkata           1
surat             1
                 ..
meerut            1
rajkot            1
kalyandombivli    1
vasaivirar        1
ahmedabad         1
Name: count, Length: 319, dtype: int64

In [14]:
cities.sample(5)

,Rank,city,population,state,city_clean,city_tier,state_clean
103,104,Gaya,385432.0,Bihar,gaya,Tier 3,bihar
46,47,Chandigarh,808515.0,Chandigarh,chandigarh,Tier 2,chandigarh
256,257,Serampore,197857.0,West Bengal,serampore,Tier 3,west bengal
303,304,Hosur,102000.0,Tamil Nadu,hosur,Tier 3,tamil nadu
98,99,Kurnool[18],345987.0,Andhra Pradesh,kurnool,Tier 3,andhra pradesh


In [15]:
# Drop rows with missing city or population

cities = cities.dropna(subset=["city_clean", "population"])

In [16]:
# Create analysis-ready dataset
cities_final = cities[
    ["city_clean", "state_clean", "population", "city_tier"]
]


In [17]:
cities_final = cities_final.drop_duplicates(subset=["city_clean"])

In [18]:
# Preview

cities_final.sample(5)

,city_clean,state_clean,population,city_tier
316,buxar,bihar,83168.0,Tier 3
4,ahmedabad,gujarat,3520085.0,Tier 1
62,jalgaon,maharashtra,462280.0,Tier 3
198,vijayanagaram,andhra pradesh,174324.0,Tier 3
288,bongaigaon,assam,101213.0,Tier 3


In [19]:
cities_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 295 entries, 0 to 320
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   city_clean   295 non-null    object 
 1   state_clean  295 non-null    object 
 2   population   295 non-null    float64
 3   city_tier    295 non-null    object 
dtypes: float64(1), object(3)
memory usage: 11.5+ KB


In [20]:
cities_final.to_csv("cities_cleaned.csv", index=False)